In [ ]:
import sys
import os
import re
import numpy as np
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import Pipeline

def load_data():

    data_dir = os.path.join("..", "data")
    df = pd.read_csv(os.path.join(data_dir, r"aavail-target.csv"))

    ## pull out the target and remove uneeded columns
    _y = df.pop('is_subscriber')
    y = np.zeros(_y.size)
    y[_y==0] = 1 
    df.drop(columns=['customer_id', 'customer_name'], inplace=True)
    return(y, df)


## variables
rs = 42

## preprocessing pipeline
numeric_features = ['age', 'num_streams']
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())])

categorical_features = ['country', 'subscriber_type']
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])

In [ ]:
if __name__ == "__main__":

    ## load the data
    y,df = load_data()
    X_train, X_test, y_train, y_test = train_test_split(df, y, test_size=0.2, stratify=y, random_state=rs)
    
    pipe  = Pipeline(steps=[('preprocessor', preprocessor),
                            ('nn', MLPClassifier(alpha=1e-5,random_state=rs))])


    param_grid = {
    'nn__activation': ['logistic', 'tanh', 'relu'],
    'nn__solver': ['lbfgs', 'sgd','adam'],
    'nn__hidden_layer_sizes': [(10,10), (50,50), (100,100), (50,50,50), (100,100,100)]
    }

    grid = GridSearchCV(pipe, param_grid=param_grid, cv=5, iid=False, n_jobs=-1)
    grid.fit(X_train, y_train)
    y_pred = grid.predict(X_test)

    ## extract feature names 
    pipe.fit(X_train, y_train)
    feature_names = np.hstack([numeric_features,
                               preprocessor.transformers_[1][1].named_steps['onehot'].get_feature_names()])
    feature_names = [re.sub("x\d_","",fn) for fn in feature_names]
    target_names = ['subscriber','churned']
    print(feature_names)
    print(classification_report(y_test, y_pred, target_names=target_names))

In [ ]:
import joblib
import time

saved_model = 'my-saved-model.joblib'
if not os.path.exists(saved_model):
    time_start = time.time()
    grid = GridSearchCV(pipe, param_grid=param_grid, cv=5, iid=False, n_jobs=-1)
    grid.fit(X_train, y_train)
    joblib.dump(grid, saved_model)
    print("train time", time.strftime('%H:%M:%S', time.gmtime(time.time()-time_start)))
else:
    print("loading {} from file".format(saved_model))
    grid = joblib.load(saved_model)

## print best parameters
print(grid.best_params_)